# Combined ETL Python Code

**Insert Dependecies**

In [1]:
import numpy as np
import pandas as pd

**Import Urbanization Index Data**

In [2]:
urban = "Resources/urban.csv"
urban_df = pd.read_csv(urban)
urban_df = pd.DataFrame(urban_df)

In [24]:
state_new  = {'Alabama': 'AL','Alaska': 'AK','American Samoa': 'AS','Arizona': 'AZ','Arkansas': 'AR',
              'California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','District of Columbia': 'DC',
              'Florida': 'FL','Georgia': 'GA','Guam': 'GU','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL',
              'Indiana': 'IN','Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME',
              'Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS',
              'Missouri': 'MO','Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH',
              'New Jersey': 'NJ','New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND',
              'Northern Mariana Islands':'MP','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA',
              'Puerto Rico': 'PR','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN',
              'Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virgin Islands': 'VI','Virginia': 'VA',
              'Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'
}

urban_df['abbrev'] = urban_df.state.replace(state_new)

In [4]:
urban = urban_df[['abbrev', 'urbanindex']].copy()
urban = urban.rename(columns={'abbrev':'State','urbanindex':'Urban Index'})

In [5]:
clean_urban = urban.drop(index = [2,37,50,12,42])

In [11]:
clean_urban['Urban Scale']= pd.qcut(clean_urban['Urban Index'], 4, 
                                    labels=["Very Rural", "Rural", 
                                            "Urban", "Very Urban"])
clean_urban = clean_urban.round(3)
clean_urban.head()

**Import Ufo Sighting Data**

In [8]:
ufodata = "Resources/scrubbed_ufo.csv"
ufodf = pd.read_csv(ufodata, low_memory=False)

In [ ]:
datetimefix = ufodf['datetime']
datetimefix = pd.DataFrame(datetimefix.str.split("/",expand=True))
datetimefix = datetimefix[2]
datetimefix = pd.DataFrame(datetimefix.str.split(expand=True))
datetimefix = pd.to_numeric(datetimefix[0])
datetimefix = pd.DataFrame(datetimefix)

In [ ]:
ufodf = ufodf.join(datetimefix, how='left')
ufodf = ufodf.rename(columns={0: "year"})

In [ ]:
ufodf = ufodf.drop(ufodf[ufodf['year'] <= 2010].index)
ufodf = ufodf.drop('year', axis = 1)
ufodf.head()

**Create Population Table**

In [14]:
states = "Resources/nst-est2019-alldata.csv"
statesdf = pd.read_csv(states, low_memory=False)

In [16]:
statespopdf1 = statesdf[["REGION", "DIVISION", "NAME"]]

In [17]:
statespopdf2 = statesdf[["POPESTIMATE2010", 
                         "POPESTIMATE2011", "POPESTIMATE2012", "POPESTIMATE2013", 
                         "POPESTIMATE2014", "POPESTIMATE2014", "POPESTIMATE2015", 
                         "POPESTIMATE2016", "POPESTIMATE2017", "POPESTIMATE2018",
                        "POPESTIMATE2019"]]


In [20]:
statespopdf2['avgpop'] = statespopdf2.mean(axis=1).astype('int64')

/Users/hijodeagua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
statespopdf = statespopdf1.join(statespopdf2)
statespopdf = statespopdf[["REGION", "DIVISION", "NAME", "avgpop"]]

In [26]:
statespopdf['abbrev'] = statespopdf.NAME.replace(state_new)
statespopdf = statespopdf.iloc[5:-1]

In [27]:
statespopdf = statespopdf[['abbrev', 'avgpop']].copy()
statespopdf = statespopdf.rename(columns={'abbrev':'State','avgpop':'Population'})
statespopdf.head()

,State,Population
5,AL,4844999
6,AK,732859
7,AZ,6798170
8,AR,2973257
9,CA,38616261


**Assign States to Regions**

In [28]:
census = "Resources/nst-est2019-alldata.csv"
censusdf = pd.read_csv(census, low_memory=False)
censusdf = pd.DataFrame(censusdf)

In [29]:
ceny = censusdf[["REGION", "DIVISION", "NAME"]]
ceny = ceny.rename(columns={'NAME':'State'})
ceny = ceny.drop(index = [0,1,2,3,4,56])

In [30]:
ceny["REGION"] = pd.to_numeric(ceny["REGION"])
ceny["DIVISION"] = pd.to_numeric(ceny["DIVISION"])

In [31]:
Regions = [
    (ceny['REGION'] <= 1),
    (ceny['REGION'] <= 2),
    (ceny['REGION'] <= 3),
    (ceny['REGION'] <= 4)
    ]
values = ['Northeast', 'Midwest', 'South', 'West']
ceny['Regions'] = np.select(Regions, values)
Divisions = [
    (ceny['DIVISION'] <= 1),
    (ceny['DIVISION'] <= 2),
    (ceny['DIVISION'] <= 3),
    (ceny['DIVISION'] <= 4),
    (ceny['DIVISION'] <= 5),
    (ceny['DIVISION'] <= 6),
    (ceny['DIVISION'] <= 7),
    (ceny['DIVISION'] <= 8),
    (ceny['DIVISION'] <= 9),
    ]
names = ['New England', 'Middle Atlantic', 'East North Central', 
          'West North Central','South Atlantic', 'East South Central',
          'West South Central', 'Mountain', 'Pacific']
ceny['Divisions'] = np.select(Divisions, names)

In [32]:
census_regions = ceny[["State", "Regions", "Divisions"]]
census_regions = census_regions.rename(columns={"Regions":'Region',"Divisions":"Division"})
census_regions = census_regions.set_index(['State'])
census_regions.head()

,Region,Division
State,,
Alabama,South,East South Central
Alaska,West,Pacific
Arizona,West,Mountain
Arkansas,South,West South Central
California,West,Pacific


**Turn New Tables into CSVs**

In [10]:
clean_urban.to_csv("tables/clean_urban.csv")
ufodf.to_csv("tables/limited_ufo.csv")
statespopdf.to_csv("tables/state_populations.csv")
census_regions.to_csv("Resources/census_regions.csv")